In [43]:
import pandas as pd
import numpy as np

data = pd.read_csv('MNIST_15_15.csv', header = None)
labels = pd.read_csv('MNIST_LABEL.csv', header = None)

def SolverLinearRegression(X, y):
    return np.dot(np.dot(np.linalg.pinv(np.dot(X.transpose(), X)), X.transpose()), y)

In [44]:
norm_data = data / 255
#setting up k-fold cv

#set number k for k-folds
k = 10

#split data set evenly into k sets
 #get shape of the folds
f_rows, f_cols = norm_data.shape
 #get correct number of rows, and round up to nearest whole number
f_rows = math.ceil(f_rows/k)

In [45]:

tpr     = [0,0,0,0,0,0,0,0,0,0]
fpr     = [0,0,0,0,0,0,0,0,0,0]
scores  = [0,0,0,0,0,0,0,0,0,0]
for i in range(k):
    
    acc = 0         #accuracy, resets with each iteration
    acc_sum = 0     #accuracy sum for using in a nested loop, will be used to calculate average of accuracy per fold loop
    
    #choose validation fold
    xi = norm_data.iloc[(f_rows * i):f_rows*(i+1), :]
    n, p = xi.shape
    X_test = xi.iloc[:,0:-1]
    X_test = pd.DataFrame(np.c_[np.ones(n), X_test])

    y_groundtruth = np.zeros(n)
    yi = labels.iloc[(f_rows * i):f_rows*(i+1), :]
    y_groundtruth[yi.iloc[:,0] > 5] = 1
    
    for j in range(k):
        #skip if same as validation fold
        if(j == i):
            continue
        #choose train fold
        xj = norm_data.iloc[(f_rows * j):f_rows*(j+1), :]
        x = xj.iloc[:,0:-1]
        n, p = x.shape
        x = pd.DataFrame(np.c_[np.ones(n), x])

        y = np.zeros(n)
        yj = labels.iloc[(f_rows * j):f_rows*(j+1), :]
        y[yj.iloc[:,0] > 5] = 1
        
        b_opt = SolverLinearRegression(x, y)
        
        #for calculating TPR and FPR
        predictions = np.dot(X_test, b_opt) > 0.5
        p_truth = (np.dot(X_test, b_opt) > 0.5) == y_groundtruth
        t_pos = 0
        c_pos = 0
        f_pos = 0
        c_neg = 0
        for m in range(len(predictions)):
            if(p_truth[m] == True):
                t_pos = t_pos + 1
            elif(y_groundtruth[m] == 0 and predictions[m] == True):
                f_pos = f_pos + 1
            if(y_groundtruth[m] == 0):
                c_neg = c_neg + 1
            elif(y_groundtruth[m] == 1):
                c_pos = c_pos + 1          
        tpr[i] = t_pos / c_pos
        fpr[i] = f_pos / c_neg
        
        #calculate accuracy and add to sum
        acc = sum(np.array(np.dot(X_test, b_opt) > 0.5) == y_groundtruth) / len(y_groundtruth)
        acc_sum = acc_sum + acc
        
    acc = acc_sum / (k-1) #calculates accuracy for each fold
    
    scores[i] = acc # array holding accuracies, will be used to find fold with max accuracy
    print("")
    print("Fold #" + str(i) + ": ")
    print("Accuracy: " + str(acc))
    print("TPR: " + str(tpr[i]))
    print("FPR: " + str(fpr[i]))
    

best = scores.index(max(scores))
print("")
print("Fold " + str(best) + " had the best score!")
print("")
print("Average Accuracy per Fold: " + str(scores))
print("")
print("TPR: " + str(tpr))
print("")
print("FPR: " + str(fpr))
print("")
print("Average Accuracy Overall:  " + str(np.average(scores)))


Fold #0: 
Accuracy: 0.8039215686274509
TPR: 1.7058823529411764
FPR: 0.17647058823529413

Fold #1: 
Accuracy: 0.8464052287581699
TPR: 1.5238095238095237
FPR: 0.0

Fold #2: 
Accuracy: 0.9150326797385622
TPR: 2.5833333333333335
FPR: 0.09090909090909091

Fold #3: 
Accuracy: 0.934640522875817
TPR: 2.1333333333333333
FPR: 0.05263157894736842

Fold #4: 
Accuracy: 0.8954248366013073
TPR: 1.4090909090909092
FPR: 0.16666666666666666

Fold #5: 
Accuracy: 0.84640522875817
TPR: 1.4736842105263157
FPR: 0.2

Fold #6: 
Accuracy: 0.8954248366013071
TPR: 1.4761904761904763
FPR: 0.07692307692307693

Fold #7: 
Accuracy: 0.9411764705882353
TPR: 2.357142857142857
FPR: 0.0

Fold #8: 
Accuracy: 0.9052287581699348
TPR: 1.7647058823529411
FPR: 0.17647058823529413

Fold #9: 
Accuracy: 0.8199233716475096
TPR: 1.5625
FPR: 0.07692307692307693

Fold 7 had the best score!

Average Accuracy per Fold: [0.8039215686274509, 0.8464052287581699, 0.9150326797385622, 0.934640522875817, 0.8954248366013073, 0.84640522875817, 